# Load Package

In [ ]:
# Check and install required packages
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = {
    'pandas': 'For data manipulation and analysis',
    'numpy': 'For numerical computing',
    'xarray': 'For working with labeled multi-dimensional arrays',
    'dask': 'For parallel computing capabilities',
    'xbatcher': 'For batch processing of large datasets',
    'matplotlib': 'For data visualization',
    'torch': 'For deep learning',
}

# Check and install missing packages
for package, purpose in required_packages.items():
    try:
        __import__(package)
        print(f"✓ {package} is already installed ({purpose})")
    except ImportError:
        print(f"Installing {package} ({purpose})...")
        
        install_package(package)
        print(f"✓ {package} has been installed")


# Verify installations
import os

# Data Analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
## Xarray and Extensions
import xarray as xr
import xbatcher as xb
import xbatcher.loaders.torch
import dask

# Deep Learning (torch)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split


import psutil
import math
import dask
import dask.array as da

from dask.distributed import Client, LocalCluster, progress, performance_report
from dask.diagnostics import ProgressBar

import time

print("\nAll required packages are installed and imported successfully!")

# Data Loading and Processing

### Step 1. Transfer to Xarray - DataArray and Labeling the dims and coords.

In [ ]:
# ls '/home/NAS/homes/isaac-10009/Data/Academic/AtmSci7115 Climate change and extreme events – machine learning hands-on/data/'

In [1]:
# root path
root_path = '/home/NAS/homes/isaac-10009/Data/Academic/AtmSci7115 Climate change and extreme events – machine learning hands-on/data/topic1'

In [ ]:
# ### Training data
# TRAINING_NUM = 35226

# # features are stored in a numpy array with shape (35226, 64, 64, 2) 
# # load the features from npz file and open
# features_path = os.path.join(root_path, f'[7C40]features_({TRAINING_NUM}, 64, 64, 2).npz')

# X_train = xr.DataArray(
#     np.load(features_path)['arr_0'],
#     dims=['time', 'height', 'width', 'channel'],  # Rename dimensions as needed
#     name='train_x',
#     coords={
#         'time': range(TRAINING_NUM),
#         'height': range(64),
#         'width': range(64),
#         'channel': ['feature0', 'feature1']  # Rename channels as needed
#     }
# )

# # labels are stored in a numpy array with shape (35226,)
# # load the label npz file and open
# labels_path = os.path.join(root_path, f'[7C40]labels_({TRAINING_NUM},).npz')
# y_train = xr.DataArray(
#     np.load(labels_path)['arr_0'],
#     dims=['time'],
#     name='train_y',
#     coords={'time': range(TRAINING_NUM)}
# )

# print(f"X_train shape: {X_train.shape}")
# print(f"y_train shape: {y_train.shape}")

In [ ]:
# ### Test data
# TESTING_NUM = 171081


# # features are stored in a numpy array with shape (171081, 64, 64, 2) 
# # load the features from npz file and open
# features_path = os.path.join(root_path, f'[7C40]features_({TESTING_NUM}, 64, 64, 2).npz')

# X_test = xr.DataArray(
#     np.load(features_path)['arr_0'],
#     dims=['time', 'height', 'width', 'channel'],  # Rename dimensions as needed
#     name='test_x',
#     coords={
#         'time': range(TESTING_NUM),
#         'height': range(64),
#         'width': range(64),
#         'channel': ['feature0', 'feature1']  # Rename channels as needed
#     }
# )

# # labels are stored in a numpy array with shape (171081,)
# # load the label npz file and open
# labels_path = os.path.join(root_path, f'[7C40]labels_({TESTING_NUM},).npz')
# y_test = xr.DataArray(
#     np.load(labels_path)['arr_0'],
#     dims=['time'],
#     name='test_y',
#     coords={'time': range(TESTING_NUM)}
# )

# print(f"X_test shape: {X_test.shape}")
# print(f"y_test shape: {y_test.shape}")

#### New Versiom with Dask to faster Loading

In [ ]:
# Set OS parms
CORE = psutil.cpu_count(logical=False)
TOTAL_MEMORY = psutil.virtual_memory().total / (1024**3) 

core_per_worker = CORE - 2
memory_limit_per_core = math.floor((TOTAL_MEMORY / CORE-1) * 0.8)

# Set dask cluster
cluster = LocalCluster(
    n_workers=core_per_worker,
    threads_per_worker = 4,
    memory_limit=f'{memory_limit_per_core}GB',
    # dashboard_address=':0',
    dashboard_address=42295,
    silence_logs='error',  
    lifetime='1h',     # 設置生命週期
    processes=False,     # 使用多進程而不是線程
)

client = Client(cluster)
print(client.dashboard_link)

In [ ]:
print("Cluster information:\n")
print(f"Number of workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {client.cluster.scheduler.total_nthreads}")
print(f"Total memory: {client.cluster.scheduler.total_nthreads} GB")

In [ ]:
### Training data
TRAINING_NUM = 35226

# Measure performance
start_time = time.time()

# Set chunk sizes for optimal performance
# Chunk along time dimension for best parallel processing
# chunks = {'time': 1000, 'height': -1, 'width': -1, 'channel': -1} 

# Load features with dask - use delayed loading
print("Loading feature data with Dask...")
features_path = os.path.join(root_path, f'[7C40]features_({TRAINING_NUM}, 64, 64, 2).npz')
# np_data = np.load(features_path)['arr_0']
# Convert to dask array with appropriate chunking
# dask_features = da.from_array(np_data, chunks=(5000, 64, 64, 2))
with ProgressBar():
    lazy_load = dask.delayed(np.load)(features_path)
    dask_features = da.from_delayed(
        lazy_load['arr_0'],
        shape=(TRAINING_NUM, 64, 64, 2),
        dtype=np.float32
    )

    # Create xarray DataArray with dask array
    X_train = xr.DataArray(
        dask_features,
        dims=['time', 'height', 'width', 'channel'],
        name='train_x',
        coords={
            'time': range(TRAINING_NUM),
            'height': range(64),
            'width': range(64),
            'channel': ['feature0', 'feature1']
        }
    )
print(f"X_train shape: {X_train.shape}, chunks: {X_train.chunks}")

# Load labels with dask
print(f"\nTrain feature loading completed in {time.time() - start_time:.2f} seconds\nLoading Train label data with Dask...")

with ProgressBar():
    labels_path = os.path.join(root_path, f'[7C40]labels_({TRAINING_NUM},).npz')
    lazy_load = dask.delayed(np.load)(labels_path)
    dask_labels = da.from_delayed(
        lazy_load['arr_0'],
        shape=(TRAINING_NUM,),
        dtype=np.float32
    )

    # Create xarray DataArray with dask array
    y_train = xr.DataArray(
        dask_labels,
        dims=['time'],
        name='train_y',
        coords={
            'time': range(TRAINING_NUM),
        }
    )

# Print information about the data
print(f"y_train shape: {y_train.shape}, chunks: {y_train.chunks}")
# y_train.compute()

# Report loading time
print(f"\nTrain Data loading completed in {time.time() - start_time:.2f} seconds")


In [ ]:
print(type(X_train.data))  # if dask.array.Array that use Dask

In [ ]:
### Test data
TESTING_NUM = 171081

# Measure performance
start_time = time.time()

# features are stored in a numpy array with shape (171081, 64, 64, 2) 
# Load features with dask - use delayed loading
print("Loading feature data with Dask...")
features_path = os.path.join(root_path, f'[7C40]features_({TESTING_NUM}, 64, 64, 2).npz')
# np_data = np.load(features_path)['arr_0']
# Convert to dask array with appropriate chunking
# dask_features = da.from_array(np_data, chunks=(1000, 64, 64, 2))


with ProgressBar():
    lazy_load = dask.delayed(np.load)(features_path)
    dask_features = da.from_delayed(
        lazy_load['arr_0'],
        shape=(TESTING_NUM, 64, 64, 2),
        dtype=np.float32
    )

    X_test = xr.DataArray(
        dask_features,
        dims=['time', 'height', 'width', 'channel'],  # Rename dimensions as needed
        name='test_x',
        coords={
            'time': range(TESTING_NUM),
            'height': range(64),
            'width': range(64),
            'channel': ['feature0', 'feature1']  # Rename channels as needed
        }
    )

print(f"X_test shape: {X_test.shape}, chunks: {X_test.chunks}")

# labels are stored in a numpy array with shape (171081,)
# Load labels with dask
print(f"\nTest feature loading completed in {time.time() - start_time:.2f} seconds\nLoading Test label data with Dask...\n")
labels_path = os.path.join(root_path, f'[7C40]labels_({TESTING_NUM},).npz')


with ProgressBar():
    lazy_load = dask.delayed(np.load)(labels_path)
    dask_labels = da.from_delayed(
        lazy_load['arr_0'],
        shape=(TESTING_NUM,),
        dtype=np.float32
    )

    y_test = xr.DataArray(
        dask_labels,
        dims=['time'],
        name='test_y',
        coords={'time': range(TESTING_NUM)}
    )

print(f"y_test shape: {y_test.shape}, chunks: {y_test.chunks}")

print(f"\nTest Data loading completed in {time.time() - start_time:.2f} seconds")


In [ ]:
print(type(X_test.data))

In [ ]:
X_train.isel(time=0).sel(channel='feature1').plot()

### Step 2. Concat to Xarray - Dataset that makes it mange easy. 

In [ ]:
ds = xr.Dataset(
    data_vars={
        'features': X_train,  
        'labels': y_train    
    }
).transpose('time', 'channel', 'height', 'width')  # Transpose dimensions to Pytorch format

# ds.persist() # Persist data in memory for faster access
ds

In [ ]:
ds_test = xr.Dataset(
    data_vars={
        'features': X_test,  
        'labels': y_test,    
    }
).transpose('time', 'channel', 'height', 'width') #(B, C, H, W) torch format

ds_test 

### Step 3. Use Xbatcher to batch generator from xarray data 

In [ ]:
# define the size
n_timepoint_in_each_sample = 1
n_timepoint_in_each_batch = 32
# input_overlap = 3 # overlap 3 time points 1-3 2-4 3-5 4-6

# Define batch generators
X_bgen = xb.BatchGenerator(
    ds['features'],
    input_dims = {'height':64, 'width':64, 'channel':2}, # don't put time dimension here (one time)
    batch_dims = {'time': n_timepoint_in_each_batch}, # batch size dims (name) can't be same to input dims
    preload_batch=True,
)

y_bgen = xb.BatchGenerator(
    ds['labels'], 
    input_dims={}, 
    batch_dims={'time': n_timepoint_in_each_batch}, 
    preload_batch=True
)



In [ ]:
# test first one batch generator
X_bgen[0]


In [ ]:
# show the first three batch size shape
for i, (batch_X, batch_y) in enumerate(zip(X_bgen, y_bgen)):

    print(f"Batch {i} shapes: X={batch_X.shape}, y={batch_y.shape}")
    if i >= 2:  
        break 

In [ ]:
print(f"Total number of batches: {len(X_bgen)}")
print(f"First train batch shape: {X_bgen[0].shape}")

### Step 4. Transfer to DataLoader from xb.batch.generator 

In [ ]:
# Map batches to a PyTorch-compatible dataset

## Train dataset (Ready to transfer to PyTorch.Dataloader)
train_dataset = xbatcher.loaders.torch.MapDataset(X_bgen, y_bgen)

# set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# cal val size
val_size = 0.2  # use 20% of the data for validation
n_samples = len(train_dataset)
n_val = int(n_samples * val_size)
n_train = n_samples - n_val

# split the dataset 
train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, 
    [n_train, n_val]
)


In [ ]:
# Test dataset (same the train_dataset: we neet to transfer to torch)
X_bgen_test = xb.BatchGenerator(
    ds_test['features'],
    input_dims = {'height':64, 'width':64, 'channel':2}, # don't put time dimension here (one time)
    batch_dims = {'time': 4096}, # batch size dims (name) can't be same to input dims (4096 is 2^12 for better performance) 
    preload_batch=True,
)

y_bgen_test = xb.BatchGenerator(
    ds_test['labels'], 
    input_dims={}, 
    batch_dims={'time': 4096}, 
    preload_batch=True
)

test_dataset = xbatcher.loaders.torch.MapDataset(X_bgen_test, y_bgen_test)

In [ ]:
# create dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=None, 
    shuffle=True,   # shuffle the data
    
    prefetch_factor=3,  # Prefetch up to 3 batches in advance to reduce data loading latency
    num_workers=4,  # Use 4 parallel worker processes to load data concurrently
    persistent_workers=True,  # Keep workers alive between epochs for faster subsequent epochs
    multiprocessing_context='forkserver',  # Use "forkserver" to spawn subprocesses, ensuring stability in multiprocessing
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=None,
    shuffle=False,  # validation data should not be shuffled

    prefetch_factor=3,  # Prefetch up to 3 batches in advance to reduce data loading latency
    num_workers=4,  # Use 4 parallel worker processes to load data concurrently
    persistent_workers=True,  # Keep workers alive between epochs for faster subsequent epochs
    multiprocessing_context='forkserver',  # Use "forkserver" to spawn subprocesses, ensuring stability in multiprocessing
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=None,
    shuffle=False,  # validation data should not be shuffled

    prefetch_factor=3,  # Prefetch up to 3 batches in advance to reduce data loading latency
    num_workers=4,  # Use 4 parallel worker processes to load data concurrently
    persistent_workers=True,  # Keep workers alive between epochs for faster subsequent epochs
    multiprocessing_context='forkserver',  # Use "forkserver" to spawn subprocesses, ensuring stability in multiprocessing
)
    

In [ ]:
train_features, train_labels = next(iter(train_dataloader))

print(f'First check Feature batch shape: {train_features.size()}, type: {train_features.dtype}')
print(f'First check Labels batch shape: {train_labels.size()}, type: {train_labels.dtype}')

In [ ]:
val_features, val_labels = next(iter(val_dataloader))

print(f'First check Feature batch shape: {val_features.size()}, type: {val_features.dtype}')
print(f'First check Labels batch shape: {val_labels.size()}, type: {val_labels.dtype}')

In [ ]:
test_features, test_labels = next(iter(test_dataloader))

print(f'First check Feature batch shape: {test_features.size()}, type: {test_features.dtype}')
print(f'First check Labels batch shape: {test_labels.size()}, type: {test_labels.dtype}')

# Training 

In [ ]:
import wandb
try:
    wandb.finish()
except:
    pass

In [ ]:
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(
    project='image-classification',  # 替換成您的項目名稱
    name='binary-classification-1',    # 替換成您的實驗名稱
    log_model=True
)

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import Accuracy, Precision, Recall, F1Score
import torchvision.models as models

class SimpleClassifier(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        
        # 使用預訓練的 ResNet18
        self.model = models.resnet18(weights=None)
        self.model.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False) 
        self.model.fc = nn.Linear(self.model.fc.in_features, 2) # output class 2 (binary classification)
         
        # init metrics
        self.train_acc = Accuracy(task='binary')
        self.val_acc = Accuracy(task='binary')
        self.test_acc = Accuracy(task='binary')
        self.test_precision = Precision(task='binary')
        self.test_recall = Recall(task='binary')
        self.test_f1 = F1Score(task='binary')
        
    def forward(self, x):
        x = x.float()
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy_with_logits(y_hat.squeeze(), y.float())
        acc = self.train_acc(y_hat.squeeze(), y)
        
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy_with_logits(y_hat.squeeze(), y.float())
        acc = self.val_acc(y_hat.squeeze(), y)
        
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=10
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            },
        }

In [ ]:
# set checkpoint callback and save path
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='../log/checkpoints/',
    filename='model-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=3,
    mode='min',
)

In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    accelerator='gpu',
    devices=1, 
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
    enable_checkpointing=True,
    enable_progress_bar=True,
    enable_model_summary=True,
)

In [ ]:
model = SimpleClassifier()
    

In [ ]:
trainer.fit(
    model=model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
# 測試模型
trainer.test(dataloaders=test_dataloader)

In [ ]:
# Finally close the dask cluster
client.close()
cluster.close()

In [10]:

import torch
from torch import nn
from torch.nn import CrossEntropyLoss
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()


In [11]:
input

tensor([[-0.8087, -0.3754, -0.3517,  0.9705, -0.3876],
        [-1.2446,  0.6229, -1.4622,  0.2435,  0.3064],
        [ 0.2169, -1.5220, -1.1998, -0.7979, -0.8019]], requires_grad=True)

In [12]:
target

tensor([0, 4, 1])

In [13]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

In [14]:
input

tensor([[-0.8683, -0.3490,  0.2707,  0.8888, -1.4920],
        [ 1.3720,  0.2165, -1.1045,  0.7531, -2.0253],
        [-0.3294, -0.0216, -0.6716, -0.0613,  0.2951]], requires_grad=True)

In [15]:
target

tensor([[0.1091, 0.0396, 0.6580, 0.0519, 0.1414],
        [0.5075, 0.1341, 0.0324, 0.2645, 0.0615],
        [0.6403, 0.1165, 0.0357, 0.1892, 0.0182]])